# 📊 Stock Performance Data Collection with yFinance

This notebook collects historical stock price data for selected S&P 500 companies using the **yfinance** API.
The goal is to build a clean, structured dataset that can later be used for:
- Financial analysis
- Visualization in Excel or Tableau
- Return and volatility calculations
- Event-based analysis

**Time period:** January 1, 2025 to July 1, 2025  (First Half of 2025)
**Sectors covered:** Technology and Banking